<a href="https://colab.research.google.com/github/LeeJZh/A_Network_That_Trains_At_Zero_Init/blob/master/network_trains_at_zero_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms

In [2]:
mnist_set = datasets.MNIST(root="./mnist_data", train=True, transform=transforms.ToTensor(), download=True)
mnist_test_set = datasets.MNIST(root="./mnist_data", train=False, transform=transforms.ToTensor(), download=True)

In [3]:
mnist_loader = torch.utils.data.DataLoader(mnist_set, 36)
mnist_test_loader = torch.utils.data.DataLoader(mnist_test_set, 36)

In [4]:
class Selection(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.c_in = in_channels
        self.counter = 0
        self.clf = nn.Linear(in_channels, 10)
    
    def forward(self, x):
        if self.counter == -1:
            out = self.clf(x)

        elif self.counter + 10 <= self.c_in:
            print("couter: {}".format(self.counter))
            out = x[:, self.counter: self.counter+10]
            self.counter += 10
        
        else:
            out = x[:, -10:]
            self.counter = -1
        
        return out

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        c_in = 28 * 28
        c_out= 512
        linears = []
        selectors = []
        for _ in range(5):
            linears += [nn.Linear(c_in, c_out)]
            selectors += [Selection(c_out)]
            c_in, cout = c_out, c_out // 2
        
        self.clf = nn.Linear(c_out, 10)
        self.linears = nn.ModuleList(linears)
        self.selectors = nn.ModuleList(selectors)
        # self.clfs = nn.ModuleList(clfs)
        self.act = nn.LeakyReLU(0.2)
    
    def forward(self, x):
        features = [x]
        outputs = []
        for l, s in zip(self.linears, self.selectors):
            feature = l(features[-1])
            # perm = torch.randperm(feature.size(1))
            outputs += [s(feature)]
            features += [self.act(feature)]
        outputs += [self.clf(feature)]
        return outputs

net = MLP().cuda()
for name, p in net.named_parameters():
    print(name, p.size())
    torch.nn.init.constant_(p, 0.0)
    print(p.data.sum())
criterion = torch.nn.CrossEntropyLoss()
adam = torch.optim.Adam(net.parameters(), lr=3e-4)

clf.weight torch.Size([10, 512])
tensor(0., device='cuda:0')
clf.bias torch.Size([10])
tensor(0., device='cuda:0')
linears.0.weight torch.Size([512, 784])
tensor(0., device='cuda:0')
linears.0.bias torch.Size([512])
tensor(0., device='cuda:0')
linears.1.weight torch.Size([512, 512])
tensor(0., device='cuda:0')
linears.1.bias torch.Size([512])
tensor(0., device='cuda:0')
linears.2.weight torch.Size([512, 512])
tensor(0., device='cuda:0')
linears.2.bias torch.Size([512])
tensor(0., device='cuda:0')
linears.3.weight torch.Size([512, 512])
tensor(0., device='cuda:0')
linears.3.bias torch.Size([512])
tensor(0., device='cuda:0')
linears.4.weight torch.Size([512, 512])
tensor(0., device='cuda:0')
linears.4.bias torch.Size([512])
tensor(0., device='cuda:0')
selectors.0.clf.weight torch.Size([10, 512])
tensor(0., device='cuda:0')
selectors.0.clf.bias torch.Size([10])
tensor(0., device='cuda:0')
selectors.1.clf.weight torch.Size([10, 512])
tensor(0., device='cuda:0')
selectors.1.clf.bias torch.S

In [5]:
def train_one_epoch():
    for x, y in mnist_loader:
        x, y = x.cuda(), y.cuda()
        adam.zero_grad()
        x = torch.flatten(x, start_dim=1)
        y_hat_list = net(x)
        loss = 0.0

        for y_hat in y_hat_list:
            loss += criterion(y_hat, y)
    
        loss.backward()
        adam.step()
    print("loss: {:.4f}".format(loss.item()))

for i in range(3):
    # print("training epoch {}".format(i))
    train_one_epoch()



couter: 0
couter: 0
couter: 0
couter: 0
couter: 0
couter: 10
couter: 10
couter: 10
couter: 10
couter: 10
couter: 20
couter: 20
couter: 20
couter: 20
couter: 20
couter: 30
couter: 30
couter: 30
couter: 30
couter: 30
couter: 40
couter: 40
couter: 40
couter: 40
couter: 40
couter: 50
couter: 50
couter: 50
couter: 50
couter: 50
couter: 60
couter: 60
couter: 60
couter: 60
couter: 60
couter: 70
couter: 70
couter: 70
couter: 70
couter: 70
couter: 80
couter: 80
couter: 80
couter: 80
couter: 80
couter: 90
couter: 90
couter: 90
couter: 90
couter: 90
couter: 100
couter: 100
couter: 100
couter: 100
couter: 100
couter: 110
couter: 110
couter: 110
couter: 110
couter: 110
couter: 120
couter: 120
couter: 120
couter: 120
couter: 120
couter: 130
couter: 130
couter: 130
couter: 130
couter: 130
couter: 140
couter: 140
couter: 140
couter: 140
couter: 140
couter: 150
couter: 150
couter: 150
couter: 150
couter: 150
couter: 160
couter: 160
couter: 160
couter: 160
couter: 160
couter: 170
couter: 170
couter: 170

In [6]:
def acc(logit, target):
    predication = torch.argmax(logit, dim=1)
    return (predication == target).to(torch.float)

In [7]:
res = None
# net.cpu()
for x, y in mnist_test_loader:
    x, y = x.cuda(), y.cuda()
    net.eval()
    with torch.no_grad():
        x = torch.flatten(x, start_dim=1)
        if res is None:
            res = acc(net(x)[-1], y)
        else:
            res = torch.cat([res, acc(net(x)[-1], y)], dim=0)
    
print(res.mean())

    

tensor(0.9636, device='cuda:0')


In [8]:
for name, p in net.named_parameters():
    print(name, p.data.sum(), p.data.mean(), (p.data < 1e-4).to(torch.float).mean())

clf.weight tensor(0.1329, device='cuda:0') tensor(2.5948e-05, device='cuda:0') tensor(0.5002, device='cuda:0')
clf.bias tensor(-0.0396, device='cuda:0') tensor(-0.0040, device='cuda:0') tensor(0.5000, device='cuda:0')
linears.0.weight tensor(-1434.9075, device='cuda:0') tensor(-0.0036, device='cuda:0') tensor(0.5875, device='cuda:0')
linears.0.bias tensor(0.2986, device='cuda:0') tensor(0.0006, device='cuda:0') tensor(0.4434, device='cuda:0')
linears.1.weight tensor(61.3044, device='cuda:0') tensor(0.0002, device='cuda:0') tensor(0.4917, device='cuda:0')
linears.1.bias tensor(-1.7179, device='cuda:0') tensor(-0.0034, device='cuda:0') tensor(0.5605, device='cuda:0')
linears.2.weight tensor(-926.9553, device='cuda:0') tensor(-0.0035, device='cuda:0') tensor(0.6144, device='cuda:0')
linears.2.bias tensor(-3.9924, device='cuda:0') tensor(-0.0078, device='cuda:0') tensor(0.5938, device='cuda:0')
linears.3.weight tensor(-841.5667, device='cuda:0') tensor(-0.0032, device='cuda:0') tensor(0.58